___
# Ciência dos Dados - PROJETO 3 - INSPER

___
## Vitor Pereira

## Rafael Almada


___

## INTRODUÇÃO

O objetivo do projeto é criar um modelo preditivo conduzido por uma análise de dados a partir de um dataset escolhido, e a partir de variáveis qualitativas e quantitativas correlacionas, estimar o valor de uma variável qualitativa. Para tanto nós decidimos fazer uma análise preditiva da variável quantitativa renda, baseando-se no uso da PNAD 2015 (dataset utilizado no projeto 1), utilizando as variáveis explicativas que a dupla julgou relevantes: Estado de nascimento, sexo, idade, cor ou raça, grau de alfabetização, ter utilizado internet nos últimos 12 meses, estar empregado ou não e rendimento no mês da pesquisa. 
A Partir dessas variaveis pretendemos analisar a partir de uma regressão linear, a dispersão dessas variáveis explicativas em relação a renda, ou seja, o quanto essas variáveis selecionadas se relacionam com a renda de uma pessoa que respondeu a pesquisa. Feita a regressão linear vamos comparar os valores obtidos de renda ao rendimento mensal domiciliar per capita de pessoas aleatórias para validar a qualidade da predição. 

___
## MINERANDO DADOS e CARACTERÍSTICAS DO DATASET

[Detalhe aqui as características da base de dados além da análise descritiva.]

In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

In [5]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

valores=pd.read_csv('PES2015csv.zip', compression='zip', header=0, sep=',', quotechar='"')


dados=pd.DataFrame({

    'Estado' :valores.UF,
    'Sexo' :valores.V0302,
    'Idade' :valores.V8005,
    'Corouraca':valores.V0404,
    'Sabe_ler_e_escrever' :valores.V0601,
    'internet12meses' :valores.V061111,
    'empregado' :valores.V0701,
    'rendimento' :valores.V7122,
    'Rendimento_mensal_domiciliar_per_capita' :valores.V4742
    })



dados=dados[['Estado','Sexo','Idade','Corouraca','Sabe_ler_e_escrever','internet12meses','empregado','rendimento','Rendimento_mensal_domiciliar_per_capita']]

dados.Sexo=dados.Sexo.astype('category')
dados.Sexo.cat.categories=(['Masculino','Feminino'])

dados.Corouraca=dados.Corouraca.astype('category')
dados.Corouraca.cat.categories=(['Branca','Preta','Amarela','Parda','Indígena','Sem declaração'])

dados.Sabe_ler_e_escrever=dados.Sabe_ler_e_escrever.astype('category')
dados.Sabe_ler_e_escrever.cat.categories=('Sim','Não')

dados.internet12meses=dados.internet12meses.astype('category')
dados.internet12meses.cat.categories=('Sim','Não')

dados.empregado=dados.empregado.astype('category')
dados.empregado.cat.categories=('Sim','Não')

def Estados(linha):
    if linha>=11 and linha<=17:
        return 'Norte'
    if linha>=21 and linha<=29:
        return 'Nordeste'
    if linha>=31 and linha<=35:
        return 'Sudeste'
    if linha>=41 and linha<=43:
        return 'Sul'
    if linha>=50 and linha<=53:
        return 'Centro-Oeste'
    
dados.Estado=dados.Estado.apply(Estados)

#dados=dados.dropna()

dados

Esperamos trabalhar no diretório
C:\Users\vitor\Documents\DP dados


,Estado,Sexo,Idade,Corouraca,Sabe_ler_e_escrever,internet12meses,empregado,rendimento,Rendimento_mensal_domiciliar_per_capita
0,Norte,Masculino,23,Indígena,Sim,NaN,NaN,NaN,8.000000e+02
1,Norte,Feminino,23,Preta,Sim,NaN,NaN,NaN,1.150000e+03
2,Norte,Feminino,35,Indígena,Sim,NaN,NaN,NaN,7.930000e+02
3,Norte,Masculino,34,Indígena,Sim,NaN,NaN,NaN,7.930000e+02
4,Norte,Feminino,11,Indígena,Sim,NaN,NaN,NaN,7.930000e+02
5,Norte,Feminino,7,Indígena,Sim,NaN,Não,NaN,7.930000e+02
6,Norte,Feminino,4,Indígena,Não,NaN,NaN,NaN,7.930000e+02
7,Norte,Feminino,18,Indígena,Sim,NaN,NaN,NaN,7.930000e+02
8,Norte,Masculino,46,Preta,Sim,Não,NaN,NaN,2.900000e+03
9,Norte,Feminino,81,Indígena,Sim,Não,NaN,NaN,2.900000e+03


### BASE DE DADOS

### ANÁLISE DESCRITIVA

___
## MODELOS DE PREDIÇÃO

[O tópico C.PROCESSO E ESTATÍSTICAS DE VALIDAÇÃO estará dentro deste tópico. Organize-o aqui de forma clara!]

[ORGANIZE AQUI TODOS OS MODELOS DE PREDIÇÃO NECESSÁRIOS NESTE PROJETO!]

o MODELO DE PREDIÇÃO PELA MÉDIA

o MODELO DOS K VIZINHOS MAIS PRÓXIMOS (K-Nearest Neighbors Regression)

o MODELO DE REGRESSÃO LINEAR (Multiple Linear Regression)

o MODELO DE ÁRVORES DE REGRESSÃO (Decision Tree Regression)

___
## CONCLUSÃO

___
## REFERÊNCIAS BIBLIOGRÁFICAS